# Linear Regression Example 2

In this example we will examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("gs://big-data-class-bucket/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.4972677251
12.6556511492
39.5776680195
4.08262063295
587.951053968


## Setting Up DataFrame for Machine Learning 

In [8]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [11]:
output = assembler.transform(data)

In [12]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [13]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [14]:
final_data = output.select("features",'Yearly Amount Spent')

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean|  497.8329424427004|
| stddev|  79.18211525090666|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean|  502.7699284956683|
| stddev|  79.78176715535547|
|    min|  282.4712457199145|
|    max|  712.3963268096637|
+-------+-------------------+



In [18]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [19]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [20]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.779563756603743,38.659212113498064,-0.02315409148028929,61.70399527587987] Intercept: -1035.87893089


In [21]:
test_results = lrModel.evaluate(test_data)

In [22]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.61694191262427|
|-11.748171286252386|
| -5.336782936908378|
|0.13783618088854155|
|  5.329143997791675|
| 1.0179112811670166|
|-3.8653232334047516|
|  3.296847817307423|
| -7.379867390337381|
| -4.263604021153469|
|-1.1176513930678311|
| -6.492686930083607|
| -4.543931403788292|
|  -9.05439105988529|
|-2.2527620127127648|
| 3.9325234478475863|
| 17.916177438150726|
|  23.59896320060585|
| 0.4367102942190968|
| 3.0150939498936395|
+-------------------+
only showing top 20 rows



In [23]:
unlabeled_data = test_data.select('features')

In [24]:
predictions = lrModel.transform(unlabeled_data)

In [25]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 398.0234091600032|
|[30.3931845423455...|  331.677041089446|
|[30.4925366965402...| 287.8080286568229|
|[30.5743636841713...| 441.9265775771771|
|[31.0472221394875...| 387.1682551912297|
|[31.3895854806643...| 409.0516997788159|
|[31.4252268808548...| 534.6320418881667|
|[31.4459724827577...|481.58011711782115|
|[31.4474464941278...| 425.9826094855614|
|[31.5147378578019...| 494.0760920176149|
|[31.6610498227460...|417.47600497296867|
|[31.7242025238451...| 509.8805742180441|
|[31.8124825597242...| 397.3542763875855|
|[31.8279790554652...| 449.0571386068268|
|[31.8530748017465...|461.53788547506474|
|[32.0123007682454...|489.01252961811065|
|[32.0180740106320...| 339.8669333071646|
|[32.0498393904573...|455.12039367360944|
|[32.0961089938451...| 374.9617451160241|
|[32.1164001209281...|460.46990145437235|
+--------------------+------------

In [26]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.88843403709
MSE: 97.7811277058
